# Animating

In [ ]:
using Interact, Plots

It is possible to create interactive animations using `@async` to create asyncronous timers

In [ ]:
using Colors

timer = Observable(time())
@async while true
    sleep(0.1)
    timer[] = time()
end
    

@manipulate for color=["yellow", "cyan", "tomato"], n=3:20, t=timer
    h, w = 100, 100
    attributes = Dict(
        "fill" => color,
        "points" => join(["$(w/2*(1+sin(θ+t))),$(h/2*(1+cos(θ+t)))" for θ in 0:2π/n:2π], ' ')
    )
    dom"svg:svg[width=$w, height=$h]"(dom"svg:polygon"(attributes=attributes))
end

Here's an animation using Plots

In [ ]:
using Plots, Colors

In [ ]:
timer = Observable(0.0)
@async while true
    sleep(0.1)
    timer[] = timer[]+0.1
end

@manipulate for col1 = colorant"red", col2 = colorant"blue", lw=1:0.1:10, t=timer
    x = -π:0.1:π
    plot(x, [sin.(x.+t) cos.(x.+2t)], color = [col1 col2], linewidth = lw)
end

If you used Interact to come up with something cool, do let us know by commenting on [this issue](https://github.com/JuliaLang/Interact.jl/issues/36). :)